# Competency Test Data Analysis

In [1]:
# imports
import os
import json  
import pandas as pd 
pd.options.display.float_format = '{:20,.2f}'.format
import numpy as np
from datetime import datetime, date, timedelta

# import functions for processing metadata
import functions as f  

# UPDATE: location that stores all .metadata files from experiemnt
competency_dir = '/Users/pranav/Documents/CMU_Research/09 ASIST/study-1_2020.08'

In [2]:
# Populate identifiers for different message and topic types
# see message_summary.md for details: 
# https://drive.google.com/drive/u/1/folders/1QqY861SUTdQkCP2WelJbm4WNTOnlk97A
competencyTask_identifier = '"topic":"observations/events/competency/task"'

In [3]:
# Create dataframe with task completion times: participant x task_id 
df = pd.DataFrame(index=range(0,16))

for subdir, dirs, files in os.walk(competency_dir):
  for file in files:
    filepath = subdir + os.sep + file 
    if 'Competency' in filepath:
        messages = f.loadMetadata(filepath)
        participantID = file.split('_')[-2].split('-')[1] # OR f.getSubject(messages)
        timing = f.competencyTestState(messages, competencyTask_identifier)
        df = pd.concat([df, timing['timeSpent']], axis=1)
        df = df.rename(columns={'timeSpent': participantID})

df.index.name = 'task_id'
df = df.loc[1:14]
df.loc['total'] = df.sum()

# manually add 2 tasks times for rescue skills 
# saving victims is homogeneous and predetermined in this version 
df.loc[15] = 7.5
df.loc[16] = 15

df = df.reindex(sorted(df.columns), axis=1)
print(df.shape)

(17, 75)


In [4]:
# read skill requirements for each subtask (external file; manually created)
skill_req = pd.read_csv('competency_data_analysis.csv')

skill_req = skill_req.iloc[:-1]
skill_req['task_id'] = skill_req['task_id'].astype(float)

# prep participant completion times
dt = df.drop('total').reset_index()
dt['task_id'] = dt['task_id'].astype(float)

# create system of equations
# 4 eqns are generated by collapsing competency tasks times by unique search skill combos assessed in each task 
x = pd.merge(skill_req, dt, on='task_id').set_index('task_id')
x = x.drop(columns=['task_name']).apply(pd.to_numeric)
eq = x.groupby('unique_skills_assessed')[x.columns[1:]].sum()  
eq = eq.loc[1:] # drop redundant col

score = pd.DataFrame(columns=eq.columns[0:6])
for p in eq.columns[6:]:
  score.loc[p] = np.linalg.solve(eq[eq.columns[0:6]].to_numpy(dtype=np.float),eq[p].to_numpy(dtype=np.float))
score = score.T
score.loc['aggregate_time'] = df.loc['total']

score.to_csv('competency_skill_estimates.csv', index=True)
score

,100,101,102,26,27,28,30,31,32,33,...,90,91,92,93,94,95,96,97,98,99
physical_search_walk,0.25,0.25,0.27,0.25,0.29,0.28,0.31,0.39,0.28,0.29,...,0.27,0.23,0.34,0.28,0.21,0.31,0.26,0.25,0.29,0.24
physical_search_obstacle,0.12,0.36,0.30,0.83,-0.13,0.24,0.30,-0.19,0.43,0.06,...,0.33,0.35,-0.12,-0.08,-0.01,0.87,-0.02,0.08,0.34,0.27
cognitive_search_pathing_decision,-4.56,-3.01,-1.34,7.25,-4.02,-5.15,-3.90,-7.34,2.31,0.97,...,-2.36,4.53,-10.28,-1.57,-0.80,-9.59,-8.52,-2.11,-8.06,-2.24
cognitive_search_knowledge_based,19.90,8.08,11.92,9.75,5.06,3.70,12.26,10.86,14.64,17.94,...,8.97,24.48,3.79,12.78,9.46,8.47,5.77,6.35,5.50,5.80
rescue_green,7.50,7.50,7.50,7.50,7.50,7.50,7.50,7.50,7.50,7.50,...,7.50,7.50,7.50,7.50,7.50,7.50,7.50,7.50,7.50,7.50
rescue_yellow,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,...,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00,15.00
aggregate_time,219.89,206.15,273.34,285.64,213.79,204.60,245.09,294.75,279.29,337.74,...,223.99,290.39,217.59,237.29,222.14,217.19,211.74,208.45,201.94,202.95


## Interpretation of Skill Estimates
- Each cell indicates the a skill index in terms of time impact on performance for each instance of skill is used during a task. 
- Example 1: walk = 0.36 means this player requires addition 0.36s for each step they need to take.
- Example 2: obstacle = -0.39 means this player saves 0.39s for each obstacle they jump over (may indicate they are better at jumping over 3-4 blocks as against walking 3-4 blocks)
- Note rescue skills are static by tasks design. In the future this may be varied across participants.